# Test of algorithm steps

In [ ]:
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI_copy as itf

In [2]:
def print_matrix(M: np.ndarray, prec: int=2):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    prec : floating point precision
    """
    for row in M:
        print(f"{np.array2string(row, precision=prec ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ')}")
    print('\n')

def print_data_ext_ps(V1: np.complex_, V2: np.complex_):
    """
    Function to print data on the effect of the external phaseshift
    that's used to match the phases of two given elements.
    
    Parameters
    ------
    V1 : element of the auxillary matrix `V`
    V2 : subsequent element of **V1**
    
    ---
    
    Further description:
    ---
    When it comes to the effect of the external phaseshift  `P`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x,y]`` ,  ``V2 = V[x,y+1]``
    
    ---
    When it comes to the effect of `exp(i*summa)` in `M`:
    
    - for **even** diagonals ( j=2,4... ):
    >> ``V1 = V[x-1,y-1]`` ,  ``V2 = V[x-1,y]``
    
    - for **odd** diagonals ( j=1,3... ):
    >> ``V1 = V[x+1,y+1]`` ,  ``V2 = V[x,y+1]``
    
    """
    
    eq_stat = ((np.angle(V1) - np.angle(V2)).round(10) == 0)
    eq_str = "\nThey are matching!"
    neq_str = "\nThey are NOT matching!\n"

    print('Affected elements:\t',
        "{:.2f}, {:.2f}".format(V1, V2))
    print('Corresponding angles:\t',
        "{:.2f}, {:.2f}".format(np.angle(V1), np.angle(V2)))

    if eq_stat:
        print(eq_str)
    else:
        print(neq_str)
        print('Full length of angles:\n',
            "{}, {}".format(np.angle(V1), np.angle(V2)))
        print('\nTheir difference:\t',
            "{}".format(np.angle(V1) - np.angle(V2)))

In [3]:
U = itf.random_unitary(4)

## Odd diagonals

### Finding external phaseshifts
They match the given elements' phases

In [4]:
V = np.conjugate(U.T)
# odd diags: 1,3,5...
m = U.shape[0]
j = 1

x = m-1
y = j-1
s = y+1

P = itf.external_ps(m, s, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

### Finding internal phaseshifts
$\delta$ and $\sum$

In [30]:
# looking at odd diagonal version!

# minus sign ! Note that the function custom_arctan(V1,V2) computes the arctan of -V1/V2 !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = 0

#compute the angles theta1 and theta2:
theta1 = delta-summ
theta2 = 2*summ-theta1
print('Check internal phases')
print((theta1+theta2)/2-summ)
print((theta1-theta2)/2-delta)

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
print_matrix(V)

delta:	-0.68-0.00j	 -> real: False
angle:	-180.0°
Check internal phases
0j
0j

Block of M:

[-1.79e-17-0.63j,  6.42e-17+0.78j]
[6.42e-17+0.78j, 1.79e-17+0.63j]


[ 0.53+0.02j, -0.41-0.26j, -0.63-0.26j,  0.11-0.09j]
[ 0.05+0.06j, -0.44-0.11j,  0.5 +0.06j,  0.3 -0.67j]
[-0.32-0.79j,  0.01-0.32j,  0.03-0.41j,  0.12-0.01j]
[-1.75e-17-2.46e-17j,  3.83e-01-5.53e-01j, -1.66e-01+3.00e-01j,
 -4.63e-01-4.65e-01j]




In [31]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(-0.00)
abs: 0.00, angle: -125.37°



## Even diagonals

### Finding external phaseshifts
They match the given elements' phases

In [32]:
# V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0
s = x-1

P = itf.external_ps(m, s, V[x,y], V[x-1,y])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(P,V)
# print("NOTE: {}. row was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

Affected elements:	 -0.32-0.79j, -0.03-0.07j
Corresponding angles:	 -1.95, -1.95

They are matching!


In [33]:
#My ansatz

# V = np.conjugate(U.T)
# even diags: 2,4,6...
m = U.shape[0]
j = 2

x = m-j
y = 0

P = itf.external_ps(m, 0, V[x,y], V[x-1,y]) #ToDo: Find out the logic where to put the external PS!
# print('old V:\n')
print_matrix(P)
V = np.matmul(P,V)
# print("NOTE: {}. row was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x-1,y])

[1.+2.22e-16j, 0.+0.00e+00j, 0.+0.00e+00j, 0.+0.00e+00j]
[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]
[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]
[0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]


Affected elements:	 -0.32-0.79j, -0.03-0.07j
Corresponding angles:	 -1.95, -1.95

They are matching!


### Finding internal phaseshifts
$\delta$ and $\sum$

In [34]:
x

2

In [35]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(-V[x-1,y], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = np.angle(V[x+1,y+1]) - np.angle(V[x-1,y+1]*np.cos(delta) - V[x,y+1]*np.sin(delta)) + np.pi/2

# print('\nangles to get equal by summa:\n',
#       np.angle(V[x+1,y+1]),
#       np.angle(V[x,y+1]*np.cos(delta) - V[x+1,y+1]*np.sin(delta)),'\n',
#       'summa:',summ)

# summ = 0

# might need to change into 'k' dependence
modes = [x-1, x]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])
# print_matrix(M)


# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.09-0.00j	 -> real: False
angle:	-9.923767701705308e-15°


In [36]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
print('Elements affected by e^(i*summ):\n')
print_data_ext_ps(V[x+1,y+1], V[x,y+1])

Nulled element:	Re(0.00) Im(-0.00)
abs: 0.00, angle: -11.63°

Elements affected by e^(i*summ):

Affected elements:	 0.38-0.55j, -0.27+0.39j
Corresponding angles:	 -0.96, 2.18

They are NOT matching!

Full length of angles:
 -0.9648413408772352, 2.176751312712558

Their difference:	 -3.141592653589793


### Continuation on the chosen diagonal
Without correct $\sum$ it's wrong

In [37]:
x += 1
y += 1

delta = itf.custom_arctan(-V[x-1,y], V[x,y])
print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

summ = 0

modes = [x-1, x]     # initial mode-pairs
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

# print('\nBlock of M:\n')
# print_matrix(M[x-1:x+1,x-1:x+1])

# print('old V:\n')
# print_matrix(V)
V = np.matmul(M,V)
# print("NOTE: {}. and {}. rows were affected!".format(x,x+1))
# print('new V:\n')
# print_matrix(V)

delta:	-0.61+0.00j	 -> real: False
angle:	180.0°


In [38]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(0.00) Im(0.00)
abs: 0.00, angle: 13.42°



In [39]:
print_matrix(V.round(10))

[ 0.53+0.02j, -0.41-0.26j, -0.63-0.26j,  0.11-0.09j]
[ 0.79-0.3j ,  0.3 +0.05j,  0.42-0.j  , -0.04+0.08j]
[ 0.  -0.j  ,  0.67+0.47j, -0.51-0.24j,  0.1 -0.07j]
[0.  +0.j  , 0.  +0.j  , 0.2 +0.05j, 0.67-0.71j]




## Odd diagonal 2.
Diagonal with more than one element

### Finding external phaseshifts
They match the given elements' phases

In [40]:
# odd diags: 1,3,5...
m = U.shape[0]
j = 3

x = m-1
y = j-1
s = y+1

P = itf.external_ps(m, s, V[x,y], V[x,y+1])
# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,P)
# print("NOTE: {}. column was affected!".format(j+1))
# print('new V:\n')
# print_matrix(V)

print_data_ext_ps(V[x,y],V[x,y+1])
print_matrix(V)

Affected elements:	 0.20+0.05j, 0.95+0.23j
Corresponding angles:	 0.23, 0.23

They are matching!
[ 0.53+0.02j, -0.41-0.26j, -0.63-0.26j,  0.13+0.06j]
[ 0.79-0.3j ,  0.3 +0.05j,  0.42-0.j  , -0.09+0.j  ]
[ 1.74e-17-2.75e-17j,  6.75e-01+4.67e-01j, -5.07e-01-2.35e-01j,
  1.07e-01+4.99e-02j]
[1.80e-17+8.81e-18j, 4.50e-17+1.07e-17j, 2.02e-01+4.78e-02j,
 9.52e-01+2.26e-01j]




### Finding internal phaseshifts
$\delta$ and $\sum$

In [41]:
# looking at odd diagonal version!

# minus sign !
delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta)) + np.pi/2

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])
print_matrix(M)

# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
print(np.angle(V[2][1]),np.angle(V[2][2]))

delta:	-1.36+0.00j	 -> real: False
angle:	180.0°

Block of M:

[ 0.96+0.17j, -0.2 -0.04j]
[-0.2 -0.04j, -0.96-0.17j]


[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]
[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]
[ 0.  +0.j  ,  0.  +0.j  ,  0.96+0.17j, -0.2 -0.04j]
[ 0.  +0.j  ,  0.  +0.j  , -0.2 -0.04j, -0.96-0.17j]


0.6059549859176614 -2.5356376676721317


In [42]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(-0.00)
abs: 0.00, angle: -156.86°



In [43]:
x -= 1
y -= 1

delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta)) + np.pi/2

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
print_matrix(V)
print(np.angle(V[1][0]),np.angle(V[1][1]))

delta:	0.61+0.00j	 -> real: True
angle:	0.0°

Block of M:

[-0.49+0.29j, -0.71+0.41j]
[-0.71+0.41j,  0.49-0.29j]


[ 5.26e-01+1.97e-02j,  8.50e-01+3.18e-02j, -7.95e-17-2.38e-17j,
 -9.24e-17+5.83e-17j]
[ 7.95e-01-3.02e-01j, -4.92e-01+1.87e-01j, -1.32e-16-3.67e-17j,
  6.32e-17-2.22e-16j]
[ 1.74e-17-2.75e-17j, -1.04e-17+7.78e-18j, -9.97e-01-8.01e-02j,
  3.63e-16+2.15e-17j]
[ 1.80e-17+8.81e-18j,  8.54e-18+3.45e-18j, -6.00e-17+1.38e-17j,
 -9.20e-01-3.93e-01j]


-0.36370084462751345 2.7778918089622797


In [44]:
print("Nulled element:\tRe({:.2f}) Im({:.2f})".format(V[x,y].real, V[x,y].imag))
print("abs: {:.2f}, angle: {:.2f}°\n".format(np.abs(V[x,y]), np.angle(V[x,y],True)))
# print('Elements affected by e^(i*summ)')
# print_data_ext_ps(V[x-1,y], V[x-1,y+1])

Nulled element:	Re(-0.00) Im(0.00)
abs: 0.00, angle: 143.08°



In [45]:
x -= 1
y -= 1

delta = itf.custom_arctan(V[x,y+1], V[x,y])

print("delta:\t{:.2f}\t -> real: {}\nangle:\t{}°".format(
    delta, not np.iscomplex(delta), np.angle(delta,True)))

# if k == j: summ = 0
#summ = np.angle(V[x-1,y-1]) - np.angle(V[x-1,y]*np.sin(delta) + V[x-1,y+1]*np.cos(delta))
summ = 0

# might need to change into 'k' dependence
modes = [y, y+1]    # initial mode-pairs    NOTE: need to update it to x,y dependence
M = np.eye(m, dtype=np.complex_)
M[modes[0],modes[0]] =  np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[0]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[0],modes[1]] =  np.cos(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)
M[modes[1],modes[1]] = -np.sin(delta) * np.exp(1j*summ) * np.exp(1j*np.pi/2)

print('\nBlock of M:\n')
print_matrix(M[y:y+2,y:y+2])


# print('old V:\n')
# print_matrix(V)
V = np.matmul(V,M)
# print("NOTE: {}. and {}. column were affected!".format(j,j+1))
# print('new V:\n')
# print_matrix(V)

delta:	0.55-0.00j	 -> real: False
angle:	-2.278933206924413e-15°

Block of M:

[5.10e-17+0.53j, 4.04e-17+0.85j]
[ 4.04e-17+0.85j, -5.10e-17-0.53j]




In [46]:
print(x,y)
V.round(3)

1 0


array([[-0.037+0.999j, -0.   -0.j   , -0.   -0.j   , -0.   +0.j   ],
       [ 0.   +0.j   ,  0.356+0.935j, -0.   -0.j   ,  0.   -0.j   ],
       [ 0.   +0.j   ,  0.   +0.j   , -0.997-0.08j ,  0.   +0.j   ],
       [-0.   +0.j   , -0.   +0.j   , -0.   +0.j   , -0.92 -0.393j]])

now we implement the other external phases which are shifted to the middle

In [47]:
m = 4
for j in range(2,m+1):
   # xi = np.angle(V[0][0])-np.angle(V[j-1][j-1])
    V = np.dot(V,itf.external_ps(m, j-1, V[0,0], V[j-1,j-1]))
print_matrix(V.round(10))

[-0.04+1.j,  0.  -0.j, -0.  +0.j,  0.  +0.j]
[ 0.  +0.j, -0.04+1.j, -0.  +0.j, -0.  +0.j]
[ 0.  +0.j,  0.  +0.j, -0.04+1.j, -0.  -0.j]
[-0.  +0.j, -0.  +0.j,  0.  +0.j, -0.04+1.j]




# Test of module
## <center>TODO</center>
* check if $V$ is a diagonal matrix after the decomposition $\rightarrow$ done
---
- extend module to save found phases: $\phi$, $\delta$, $\sum$ $\Rightarrow$ $\theta_1$ & $\theta_2$ $\rightarrow$ done, but maybe needs to be adapted
- check the function in module that recreates the initial unitary matrix based on the decomposed matrices (probably needs work)
- update the drawing function in the module such that it follows the updated **Clement's** `draw()` function but with sMZI:s
- Finally: sweep through the whole module and finalise *comments*, *documentation*, *variables* and implement possible *optimalisations*
---
* optional: `import numba` for faster execution; optimized machine code at runtime

In [1]:
# including imports again in case someone only wishes
# to run this part of the notebook
import numpy as np
import os

if os.getcwd()[-9:]=="notebooks":
    os.chdir("..")

import sMZI as itf

def print_matrix(M: np.ndarray, prec: int=2):
    """
    Function to print a given matrix in a nice way.
    
    Parameters
    ------
    M : matrix to print
    prec : floating point precision
    """
    for row in M:
        print(f"{np.array2string(row, precision=prec ,formatter={'float': lambda row: f'{row:.2f}'},separator=', ', suppress_small=True)}")
    print('\n')

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)


U = itf.random_unitary(5)
I = itf.square_decomposition(U)

# I.draw()

In [2]:
print("number of output phases: {}\nnumber of input phases: {}".format(np.nonzero(I.output_phases)[0].size,np.nonzero(I.input_phases)[0].size))
print("\nThe matrix encrypting the circuit after Bell's PS shifting:\n")
print_matrix(I.circuit)

number of output phases: 2
number of input phases: 2

The matrix encrypting the circuit after Bell's PS shifting:

[ 0.34+0.j, -3.05+0.j,  0.57-0.j, -2.68+0.j,  0.54+0.j]
[ 2.61-0.j, -4.04-0.j,  1.64+0.j, -1.74-0.j, -1.75+0.j]
[ 3.09-0.j, -3.49+0.j,  1.47-0.j, -3.07+0.j, -3.88-0.j]
[ 4.53+0.j, -3.69+0.j, -0.23+0.j, -0.85-0.j, -4.38+0.j]
[-0.31+0.j, -5.66+0.j,  3.11+0.j, -2.29+0.j,  0.  +0.j]




## ``Interferometer.draw()`` to be updated!